In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [5]:
!pip install -qU langchain-openai

In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-496a7e79-e7ac-4c39-9fc4-c506d32cc770-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [10]:
result = model.invoke(messages)
result

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-49501f15-263d-4970-98ad-972b1d04cef4-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

In [11]:
parser.invoke(result)

'ciao!'

In [12]:
chain = model | parser
chain

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10bdca510>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10b896610>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [13]:
chain.invoke(messages)

'Ciao!'

In [14]:
from langchain_core.prompts import ChatPromptTemplate

In [15]:
system_template = "Translate the following into {language}:"

In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [17]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

https://python.langchain.com/v0.1/docs/modules/model_io/chat/custom_chat_model/


In [18]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [19]:
chain = prompt_template | model | parser

In [20]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'

In [ ]:
# Run server.py

In [17]:
!pip install "langserve[all]"

  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.0 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 16.9 MB/s eta 0:00:00


In [18]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'Ciao'